# LangChain: Models, Prompt Templates e Output Parsers

---

In [9]:
# Importando Bibliotecas
from dotenv import load_dotenv
import openai
import os

In [10]:
#Carregando API KEY da OpenAI diretamente da minha variável de ambiente .env

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

#Selecionando Modelo de LLM da OpenAI
llm_model = "gpt-4o-mini"

---

### Trabalhando SEM o langchain

In [12]:
# Função responsável por receber o prompt e processar resposta baseada nele.
def get_completion(prompt, model=llm_model):
    messages = [{"role":"user", "content":prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

Nesse exemplo, estou trabalhando de forma "crua", sem o framework.

A ideia é fazer o modelo traduzir a minha apresentação pessoal para outro idioma, o inglês.

In [13]:
sobre_mim = """Olá! Meu nome é Diogo Dias, sou estudante de \
Engenharia pela Universidade Federal da Paraíba (UFPB) e facinado por \
tudo que envolve dados e inteligência artificial. \
Estou trabalhando para que em breve eu consiga reopcionar meu curso \
para Ciência de Dados e IA na própria UFPB, porém, enquanto isso não acontece, \
faço uma graduação à distância em Ciência de Dados pela Cruzeiro do Sul Virtual. \
Seria legal te conhecer também! Entra em contato comigo pelo meu Discord para \
trocarmos uma idéia ;) meu nick é azafama!""" 

In [14]:
traducao = """Inglês do UK"""

In [15]:
prompt = f"""Traduza o texto de apresentação pessoal \
destacado pelas três aspas invertidas \
para o seguinte idioma {traducao}.
texto: ```{sobre_mim}```"""
print(prompt)

Traduza o texto de apresentação pessoal destacado pelas três aspas invertidas para o seguinte idioma Inglês do UK.
texto: ```Olá! Meu nome é Diogo Dias, sou estudante de Engenharia pela Universidade Federal da Paraíba (UFPB) e facinado por tudo que envolve dados e inteligência artificial. Estou trabalhando para que em breve eu consiga reopcionar meu curso para Ciência de Dados e IA na própria UFPB, porém, enquanto isso não acontece, faço uma graduação à distância em Ciência de Dados pela Cruzeiro do Sul Virtual. Seria legal te conhecer também! Entra em contato comigo pelo meu Discord para trocarmos uma idéia ;) meu nick é azafama!```


In [16]:
resposta = get_completion(prompt)
print(resposta)

Sure! Here’s the translation of your text into UK English:

```Hello! My name is Diogo Dias, I am a student of Engineering at the Federal University of Paraíba (UFPB) and I am fascinated by everything that involves data and artificial intelligence. I am working towards the goal of soon being able to switch my course to Data Science and AI at UFPB itself; however, while that is not happening, I am pursuing a distance learning degree in Data Science at Cruzeiro do Sul Virtual. It would be great to meet you too! Get in touch with me on my Discord so we can have a chat ;) my username is azafama!```


---

### Instalando LangChain

In [17]:
# Pacote do LangChain para modelos da OpenAI (seguindo a documentação)
!pip install -qU langchain-openai

In [18]:
# Importando biblioteca do langchain para trabalhar com modelos da OpenAI
from langchain_openai import ChatOpenAI

In [19]:
# Escolhendo o modelo
# Temperatura pode variar de 0 a 1, isso afeta a forma com que o modelo trás as respostas. 
# Para uma resposta mais replicável, 0.0 é o ideal.
model = ChatOpenAI(model=llm_model, temperature=0.0)

---

### Prompt Templates

A ideia é fazer com que prompts consigam ser componentizados para ser reaproveitados no código. A biblioteca por padrão possui alguns templates, mas, é possível realizar a criação dos próprios.

In [20]:
# Repare que aqui não estou usando o (f) antes das aspas para formatar a string, 
# se eu adicionar (f) as variáveis (traducao, sobre_mim) não serão reconhecidas como variáveis no template_string
template_string = """Traduza o texto de apresentação pessoal \
destacado pelas três aspas invertidas \
para o seguinte idioma {traducao}.
texto: ```{sobre_mim}```"""

In [21]:
from langchain.prompts import ChatPromptTemplate

# Criando meu Prompt Template
prompt_template = ChatPromptTemplate.from_template(template_string)

In [22]:
# Extraindo o conteúdo do meu prompt Template.
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['sobre_mim', 'traducao'], template='Traduza o texto de apresentação pessoal destacado pelas três aspas invertidas para o seguinte idioma {traducao}.\ntexto: ```{sobre_mim}```')

É possível observar as variáveis de input quando se printa o prompt template.

In [23]:
# Visualizando apenas minhas variáveis do prompt_template
prompt_template.messages[0].input_variables

['sobre_mim', 'traducao']

In [24]:
idioma = """Inglês do UK"""

In [25]:
apresentacao = """Olá! Meu nome é Diogo Dias, sou estudante de \
Engenharia pela Universidade Federal da Paraíba (UFPB) e facinado por \
tudo que envolve dados e inteligência artificial. \
Estou trabalhando para que em breve eu consiga reopcionar meu curso \
para Ciência de Dados e IA na própria UFPB, porém, enquanto isso não acontece, \
faço uma graduação à distância em Ciência de Dados pela Cruzeiro do Sul Virtual. \
Seria legal te conhecer também! Entra em contato comigo pelo meu Discord para \
trocarmos uma idéia ;) meu nick é azafama!""" 

In [26]:
# Inputando valores as variáveis do prompt template
apresentacao_pessoal = prompt_template.format_messages(
    sobre_mim = apresentacao,
    traducao = idioma
)

In [27]:
print(type(apresentacao_pessoal))
print(type(apresentacao_pessoal[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [28]:
print(apresentacao_pessoal[0])

content='Traduza o texto de apresentação pessoal destacado pelas três aspas invertidas para o seguinte idioma Inglês do UK.\ntexto: ```Olá! Meu nome é Diogo Dias, sou estudante de Engenharia pela Universidade Federal da Paraíba (UFPB) e facinado por tudo que envolve dados e inteligência artificial. Estou trabalhando para que em breve eu consiga reopcionar meu curso para Ciência de Dados e IA na própria UFPB, porém, enquanto isso não acontece, faço uma graduação à distância em Ciência de Dados pela Cruzeiro do Sul Virtual. Seria legal te conhecer também! Entra em contato comigo pelo meu Discord para trocarmos uma idéia ;) meu nick é azafama!```'


In [29]:
resposta_langchain = model(apresentacao_pessoal)

c:\Users\diasd\anaconda3\envs\ChatBot\Lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


In [30]:
# Printar o retorno completo do modelo com langchain, mostra algumas curiosidades como tokens, e outros.
print(resposta_langchain)

content='Sure! Here’s the translation of your text into UK English:\n\n```Hello! My name is Diogo Dias, I am a student of Engineering at the Federal University of Paraíba (UFPB) and I am fascinated by everything that involves data and artificial intelligence. I am working towards the goal of soon being able to switch my course to Data Science and AI at UFPB itself; however, while that is not happening, I am pursuing a distance learning degree in Data Science at Cruzeiro do Sul Virtual. It would be great to meet you too! Get in touch with me on my Discord so we can have a chat ;) my username is azafama!```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 136, 'prompt_tokens': 149, 'total_tokens': 285}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8080d70943', 'finish_reason': 'stop', 'logprobs': None} id='run-cbd2aa9a-4bbe-4232-b24f-5781910703dc-0' usage_metadata={'input_tokens': 149, 'output_tokens': 136, 'total_tokens': 

In [31]:
print(resposta_langchain.content)

Sure! Here’s the translation of your text into UK English:

```Hello! My name is Diogo Dias, I am a student of Engineering at the Federal University of Paraíba (UFPB) and I am fascinated by everything that involves data and artificial intelligence. I am working towards the goal of soon being able to switch my course to Data Science and AI at UFPB itself; however, while that is not happening, I am pursuing a distance learning degree in Data Science at Cruzeiro do Sul Virtual. It would be great to meet you too! Get in touch with me on my Discord so we can have a chat ;) my username is azafama!```


---

### Output Parser 

Output Parsers são usadas para processar a saída gerada pelo LLM para convertê-la em um formato mais útil ou estruturado.

Útil quando é preciso estruturar os dados para utilização em outras aplicações.

---

Sem output parser


In [32]:
{
    "ração": "xanin",
    "tempo_de_entrega": 3,
    "preco": "Em conta!"
}

{'ração': 'xanin', 'tempo_de_entrega': 3, 'preco': 'Em conta!'}

In [45]:
feedback_cliente = """Comprei a ração whiskas para meu gato no estabelecimento \
infelizmente ele não se adaptou muito bem. Demorou bastante pra chegar aqui em casa, \
fiquei espantado, o estebelecimento é na cidade vizinha e levou uns dois dias para entregar! Absurdo! \
Sem contar que o preço é bem salgado, muito caro, se não houver melhora no serviço \
irei trocar de loja!
    """

In [46]:
template_feedback = """\

para o texto a seguir, extraia as informações: \

ração: o nome da marca da ração para animais descrita no texto. Caso não houver um nome de marca \
adicione um None.

tempo_de_entrega: a quantidade o tempo que o cliente esperou para realização \
da entrega. Caso a quantidade de tempo tenha sido escrita, coverta em número.

preco: classifique como satisfatório ou caro, de acordo com a análise do feedback do cliente.

formate essas informações em um JSON com as seguintes chaves:

racao
tempo_de_entrega
preco

texto: {feedback_cliente}
"""

In [47]:
prompt_template = ChatPromptTemplate.from_template(template_feedback)
print(prompt_template)

input_variables=['feedback_cliente'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['feedback_cliente'], template='\npara o texto a seguir, extraia as informações: \nração: o nome da marca da ração para animais descrita no texto. Caso não houver um nome de marca adicione um None.\n\ntempo_de_entrega: a quantidade o tempo que o cliente esperou para realização da entrega. Caso a quantidade de tempo tenha sido escrita, coverta em número.\n\npreco: classifique como satisfatório ou caro, de acordo com a análise do feedback do cliente.\n\nformate essas informações em um JSON com as seguintes chaves:\n\nracao\ntempo_de_entrega\npreco\n\ntexto: {feedback_cliente}\n'))]


In [48]:
messages = prompt_template.format_messages(feedback_cliente=feedback_cliente)
resposta = model(messages)
print(resposta.content)

```json
{
  "racao": "whiskas",
  "tempo_de_entrega": 2,
  "preco": "caro"
}
```


In [52]:
# O tipo da resposta sem o parse é string, ou seja, preciso converter para
# um formato onde seja possível a utilização em uma aplicação!
# é aqui onde entram os parses.
print(type(resposta.content))

<class 'str'>


In [54]:
# Importando ferramentas de parse
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [94]:
# Criação dos Schemas
racao_schema = ResponseSchema(name="racao",
                              description="A ração foi comprada para algum animal? \
                                responda com o nome da marca se sim, ou None se não.")

tempo_de_entrega_schema = ResponseSchema(name="tempo_de_entrega",
                                         description="Quantos dias levou para que o item fosse entregue \
                                          Se o dia for escrito, converta para número. Se a informação não for \
                                            encontrada, retorne -1")

preco_schema = ResponseSchema(name="preco",
                              description="Classifique o preço como caro ou acessível \
                                de acordo com a descrição do usuário. Retorne 'não informado' para feedback \
                                  de preço não informado.") 

response_schema = [racao_schema,
                   tempo_de_entrega_schema,
                   preco_schema]

In [82]:
# Estruturando a saída através dos schemas 
output_parser = StructuredOutputParser.from_response_schemas(response_schema)

In [83]:
prompt = prompt_template.format_messages(feedback_cliente=feedback_cliente)

In [84]:
resposta = model(prompt)

In [85]:
# Com base na estrutura passada para a variável output_parser, 
# é possível extrair o conteúdo da resposta do modelo 
# e moldar para uma saída filtrada.
structured_output = output_parser.parse(resposta.content)

In [86]:
print(structured_output)

{'racao': 'whiskas', 'tempo_de_entrega': 2, 'preco': 'caro'}


In [87]:
# Agora o tipo do dado foi convertido a dicionário
print(type(structured_output))

<class 'dict'>


In [89]:
structured_output.get("racao")

'whiskas'